In [ ]:
!pip install llama-cloud==0.1.4
!pip install llama-index==0.11.21
!pip install llama-index-agent-openai==0.3.4
!pip install llama-index-cli==0.3.1
!pip install llama-index-core==0.11.23
!pip install llama-index-embeddings-huggingface==0.3.1
!pip install llama-index-embeddings-openai==0.2.5
!pip install llama-index-indices-managed-llama-cloud==0.4.0
!pip install llama-index-legacy==0.9.48.post3
!pip install llama-index-llms-groq==0.3.0
!pip install llama-index-llms-huggingface==0.3.5
!pip install llama-index-llms-huggingface-api==0.2.0
!pip install llama-index-llms-llama-cpp==0.3.0
!pip install llama-index-llms-openai==0.2.16
!pip install llama-index-llms-openai-like==0.3.0
!pip install llama-index-multi-modal-llms-openai==0.2.3
!pip install llama-index-program-openai==0.2.0
!pip install llama-index-question-gen-openai==0.2.0
!pip install llama-index-readers-file==0.2.2
!pip install llama-index-readers-llama-parse==0.3.0
!pip install llama-index-vector-stores-chroma==0.4.0
!pip install llama-index-vector-stores-faiss==0.2.1
!pip install llama-parse==0.5.12
!pip install llama_cpp_python==0.2.90

ERROR: Operation cancelled by user
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50

In [ ]:
!pip install --upgrade huggingface_hub


In [ ]:
from llama_index.core.schema import TextNode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.groq import Groq
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
import json

In [ ]:
!pip install llama-index


In [ ]:
from llama_index.core.schema import TextNode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.groq import Groq
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core import Document
import json


api_key = ""
llm = Groq(model="llama-3.1-8b-instant", api_key=api_key)

def create_llama_index():
    try:
        index_dir = './index'  # Specify the directory your index will be stored

        with open('products.json', 'r', encoding='utf-8') as file:
            product_data = json.load(file)

        # Create Document objects from the product data
        nodes = [Document(text=json.dumps(product)) for product in product_data]

        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        index = VectorStoreIndex.from_documents(nodes, embed_model=embed_model)
        Settings.embed_model = embed_model

        index.storage_context.persist(persist_dir=index_dir)

        return print({'result': 'File indexed successfully'})
    except Exception as e:
        return print({'error': f"An error occurred: {e}"})

create_llama_index()

{'result': 'File indexed successfully'}


In [ ]:
!pip install llama-index

In [ ]:
!pwd

/content


In [ ]:
def query_index():
    # retrive open ai key
    try:

        index_dir = 'index'

        # data = request.get_json()
        # prompt = data.get('prompt')
        # chatHistory = data.get('chatHistory')

        prompts = [
            "Jakie są cechy zestawu wierteł i końcówek BOSCH?",
            "Czy możesz wymienić szczegóły techniczne mieszadła specjalnego FESTOOL CS 120*600 M-14?",
            "Do czego służy torba wielofunkcyjna BESSEY STE-BAG?",
            "Który produkt zawiera adapter obrotowy do ścisków stolarskich i jakie są jego cechy?",
            "Jaka jest waga i zużycie energii systemu do szlifowania na mokro TORMEK T-4?",
            "Jaki rodzaj opakowania posiada zestaw końcówek i wierteł BOSCH?",
            "Jakie materiały można transportować w torbie BESSEY STE-BAG?",
            "Jaka jest średnica i długość mieszadła specjalnego FESTOOL?",
            "Który produkt nadaje się do ostrzenia noży i dłut oraz jakie są jego kluczowe szczegóły techniczne?",
            "Które produkty w danych mają możliwość regulacji, i jakie to funkcje?"
        ]


        storage_context = StorageContext.from_defaults(persist_dir=index_dir)
        index = load_index_from_storage(storage_context)
        query_engine = index.as_query_engine(llm=llm)

        chat_engine = index.as_chat_engine(chat_mode="condense_question", llm=llm, verbose=True)

        results = []

        for prompt in prompts:
            response_node = chat_engine.chat(prompt)
            results.append({'prompt': prompt, 'result': response_node.response})

        # Wyświetlanie wyników z dwoma liniami przerwy
        for item in results:
            print(f"Prompt:\n{item['prompt']}\n")
            print(f"Result:\n{item['result']}\n")
            print("\n")

        return print({'\n\nresult': 'Query executed successfully'})

    except Exception as e:
        return print({'error':  f"An error occurred: {e}"})

query_index()

Querying with: Jakie są cechy zestawu wierteł i końcówek BOSCH?
Querying with: Czy możesz wymienić szczegóły techniczne mieszadła specjalnego FESTOOL CS 120*600 M-14, biorąc pod uwagę cechy zestawu wierteł i końcówek, które charakteryzują się wszechstronnością, trwałością i funkcjonalnym opakowaniem?
Querying with: Czy torba wielofunkcyjna BESSEY STE-BAG jest dostępna wraz z zestawem wierteł i końcówek BOSCH, a także jest kompatybilna z mieszadłem specjalnym FESTOOL CS 120*600 M-14?
Querying with: Czy zestaw wierteł i końcówek BOSCH zawiera adapter obrotowy do ścisków stolarskich i jakie są jego cechy?
Querying with: Jaka jest waga i zużycie energii systemu do szlifowania na mokro TORMEK T-4?
Querying with: Jaki jest rodzaj opakowania zestawu wierteł i końcówek BOSCH?
Querying with: Czy torba wielofunkcyjna BESSEY STE-BAG jest dostępna wraz z zestawem wierteł i końcówek BOSCH i jakie są jej cechy, a także czy jest kompatybilna z mieszadłem specjalnym FESTOOL CS 120*600 M-14?
Querying w

In [ ]:
!pwd
!ls

/content
index  sample_data
